In [ ]:
import os
import re

import numpy as np
import pandas as pd

input_folder = './'

barcode_column = 'barcode'
tag_column = 'found'
aa_column  = 'aa_mutation'
n_aa_column = 'n_aa_substitutions'
valid_barcode_tag = '1'

filter_freq = 1 # barcodes found more than this.

In [ ]:
input_file = 'c_Bin.csv'

first_time = True

for f in os.listdir(input_folder):
    if re.search("^Bin[0-9]+.csv$", f):
        
        p_bint = pd.read_csv(os.path.join(input_folder,f))

        bint_c = p_bint.groupby(p_bint.columns.tolist(),as_index=False, dropna=False).size().copy()

            
        if first_time:

            bint_c.to_csv(input_file, index = False, header = True, mode = 'w')
            first_time = False

        else:
            bint_c.to_csv(input_file, index = False, header = False, mode = 'a')

bint = pd.read_csv(input_file)

bint_freq = bint.query(f"size > {filter_freq}").copy()

tags_sorted = np.sort ( bint[tag_column].unique() )

tag_sizes = pd.DataFrame(data={'tag' : ['total'],
              'n_barcodes': [bint['size'].sum()],
              'n_unique_barcodes': [bint.shape[0]],
              'n_frequent_barcodes': [bint_freq['size'].sum()],
              'n_unique_frequent_barcodes': [bint_freq.shape[0]]
             })

for i in tags_sorted:

    bint_tag = bint.query(f"{tag_column} == {i}")
    bint_freq = bint_tag.query(f"size > {filter_freq}").copy()

    tag_sizes = tag_sizes.append(pd.DataFrame(data={'tag' : [i],
                  'n_barcodes': [bint_tag['size'].sum()],
                  'n_unique_barcodes': [bint_tag.shape[0]],
                  'n_frequent_barcodes': [bint_freq['size'].sum()],
                  'n_unique_frequent_barcodes': [bint_freq.shape[0]]
                 }))

    if str(i) == valid_barcode_tag:
        bint_freq[[aa_column,n_aa_column,'size']].fillna('wt').groupby([aa_column,n_aa_column], as_index=False).sum().to_csv('a_'+input_file, index=False)

        mutations_sorted = np.sort ( bint_tag[n_aa_column].unique() )
        mutation_sizes =  pd.DataFrame()

        for j in mutations_sorted:
            mutation_sizes = mutation_sizes.append(pd.DataFrame(data={'mutation': ["%d" % j],
                'n_mutations' : [bint_tag.query(f"{n_aa_column} == {j}")['size'].sum()],
                'n_unique_mutaions' : [bint_tag.query(f"{n_aa_column} == {j}")[aa_column].unique().shape[0]],
                'n_frequent_mutations' : [bint_freq.query(f"{n_aa_column} == {j}")['size'].sum()],
                'n_unique_frequent_mutations' : [bint_freq.query(f"{n_aa_column} == {j}")[aa_column].unique().shape[0]]
            }
            ))


tag_sizes.index.name = input_file
display(tag_sizes)
mutation_sizes.index.name = input_file
display(mutation_sizes)